# TP1 -- Approches collaboratives : utilisateur-utilisateur, item-item, et agglomérative


Lamia Salhi - 2164386

Théo Delbecq - 2161923

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

#### Charger les Données

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving votes.csv to votes.csv


In [ ]:
# votes
votes = pd.read_csv('Data/votes.csv')

In [ ]:
## Matrice Utilisateur Item
MUI = votes.pivot(index="user.id", columns="item.id", values="rating")
MUI

item.id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user.id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Question 2 - Filtres Collaboratifs

Calculez l'erreur des prédictions des approches de filtres collaboratifs item-item et utilisateur-utilisateur. Utilisez le cosinus comme mesure de similarité entre items. Procédez par étape et rapportez, à chaque étape, l'erreur quadratique moyenne et l'erreur absolue moyenne.
Répéter pour chacune des deux approches, item-item et utilisateur-utilisateur, les étapes suivantes :

### 2.1 Utilitaires

In [ ]:
#Implémentation de fonctions utilitaires pour l'évaluation

def RMSE_mat(y_pred, y_true):
    return np.sqrt(np.nanmean((y_pred - y_true)**2))

def MAE_mat(y_pred, y_true):
    return np.nanmean(np.abs(y_pred - y_true))

def cos_sim_rows(mat):
    """
    Calcul de la similarité du cosinus entre les lignes d'une même matrice
    """
    norms = np.linalg.norm(mat, axis=1)
    norms[norms == 0] = 1 #For the zero division case
    return (mat @ mat.T) / (norms[:, np.newaxis] * norms[np.newaxis, :])

### 2.2 Traitement




#### Etape a 
Prédiction avec tous les utilisateurs/items (sans voisins rapprochés) et sans correction pour biais utilisateur/item. (6 points)

In [ ]:
def etape_a(mat_train, test = False, mat_test = None, idx_train = None):
  """
  Modèle de prédiction de base par la moyenne pondérée des voisins.

  Parameter
  ---------
  mat_train : np.array
      Matrice des données d'entraînement
  test : bool
      Realisation de l'evaluation sur un ensemble de test fourni
  mat_test : np.array
      Matrice des données de test
  idx_train : np.array
      Index des cellules contenant des données d'entraînement

  Returns
  -------
  float64, float64
      RMSE et MAE sur la matrice d'entraînement ou celle de test s'il y a lieu
  """
  mat_zeros = np.nan_to_num(mat_train)

  W = cos_sim_rows(mat_zeros)

  np.fill_diagonal(W, 0)

  Kappa = np.abs(W) @ ~np.isnan(mat_train)

  Kappa[Kappa == 0] = 1

  Preds = (W @ mat_zeros) / Kappa

  if test:
    Preds[idx_train] = np.nan

    return RMSE_mat(Preds, mat_test), MAE_mat(Preds, mat_test)
  
  else:
    return RMSE_mat(Preds, mat_train), MAE_mat(Preds, mat_train)

#### Etape b 
Correction pour biais utilisateur/item. Vous pouvez prendre l'ensemble des les valeurs observées dans le calcul des moyennes items et utilisateurs (des précisions sur ce point seront données en classe — voir aussi le bonus en (f)). (4 points)

In [ ]:
def etape_b(mat_train, test = False, mat_test = None, idx_train = None):
  """
  Modèle de prédiction avec correction par les moyennes des lignes.

  Parameter
  ---------
  mat_train : np.array
      Matrice des données d'entraînement
  test : bool
      Realisation de l'evaluation sur un ensemble de test fourni
  mat_test : np.array
      Matrice des données de test
  idx_train : np.array
      Index des cellules contenant des données d'entraînement

  Returns
  -------
  float64, float64, np.array, np.array, np.array
      RMSE et MAE sur la matrice d'entraînement ou celle de test s'il y a lieu
      Les matrice de poids, de votes centrés et les moyennes sont aussi renvoyées
      pour gagner du temps sur les étapes suivantes
  """
  row_means = np.nanmean(mat_train, axis=1)

  row_means[np.isnan(row_means)] = 0

  R = mat_train - row_means[:, np.newaxis]

  R_zeros = np.nan_to_num(R)
  
  W = cos_sim_rows(R_zeros)

  np.fill_diagonal(W, 0)

  Kappa = np.abs(W) @ ~np.isnan(mat_train)

  Kappa[Kappa == 0] = 1

  Preds = (W @ R_zeros) / Kappa + row_means[:, np.newaxis]

  if test:
    Preds[idx_train] = np.nan

    return RMSE_mat(Preds, mat_test), MAE_mat(Preds, mat_test), W, R_zeros, row_means
  
  else:
    return RMSE_mat(Preds, mat_train), MAE_mat(Preds, mat_train), W, R_zeros, row_means

#### Etape c
Ajout de 100 voisins rapprochés (ou le maximum disponible jusqu'à concurrence de 100). Vous pouvez prendre le cosinus ou la distance euclidienne pour les voisins rapprochés. (2 points)

In [ ]:
def etape_c(mat_train, W, R_zeros, row_means, test = False, mat_test = None, idx_train = None):
  """
  Modèle de prédiction avec correction par la proximité des voisins.

  Parameter
  ---------
  mat_train : np.array
      Matrice des données d'entraînement
  W : np.array
      Matrice de poids de l'étape précédente
  R_zeros : np.array
      Matrice des votes centrés
  row_means : np.array
      Moyennes des lignes
  test : bool
      Realisation de l'evaluation sur un ensemble de test fourni
  mat_test : np.array
      Matrice des données de test
  idx_train : np.array
      Index des cellules contenant des données d'entraînement

  Returns
  -------
  float64, float64, np.array
      RMSE et MAE sur la matrice d'entraînement ou celle de test s'il y a lieu
      La matrice de poids modifiées et renvoyée pour les étapes suivantes
  """

  top100 = np.repeat(np.sort(W, axis=1)[:,len(W) - 100][:, np.newaxis], W.shape[1], axis=1)

  W[W < top100] = 0

  Kappa = np.abs(W) @ ~np.isnan(mat_train)

  Kappa[Kappa == 0] = 1

  Preds = (W @ R_zeros) / Kappa + row_means[:, np.newaxis]

  if test:
    Preds[idx_train] = np.nan

    return RMSE_mat(Preds, mat_test), MAE_mat(Preds, mat_test), W
  
  else:
    return RMSE_mat(Preds, mat_train), MAE_mat(Preds, mat_train), W

#### Etape d
Correction pour le nombre de votes communs. (2 points)

In [ ]:
def etape_d(mat_train, W, R_zeros, row_means, gamma = 5, test = False, mat_test = None, idx_train = None):
  """
  Modèle de prédiction avec correction par le nombre de vote commun.

  Parameter
  ---------
  mat_train : np.array
      Matrice des données d'entraînement
  W : np.array
      Matrice de poids de l'étape précédente
  R_zeros : np.array
      Matrice des votes centrés
  row_means : np.array
      Moyennes des lignes
  gamma : int
      Seuil pour la correction des votes commun
  test : bool
      Realisation de l'evaluation sur un ensemble de test fourni
  mat_test : np.array
      Matrice des données de test
  idx_train : np.array
      Index des cellules contenant des données d'entraînement

  Returns
  -------
  float64, float64, np.array
      RMSE et MAE sur la matrice d'entraînement ou celle de test s'il y a lieu
      La matrice de poids modifiées et renvoyée pour les étapes suivantes
  """

  vote_mask = ~np.isnan(mat_train)
  nb_common_votes = vote_mask @ vote_mask.T

  W = W * (np.maximum(nb_common_votes, gamma) / gamma)

  Kappa = np.abs(W) @ ~np.isnan(mat_train)

  Kappa[Kappa == 0] = 1

  Preds = (W @ R_zeros) / Kappa + row_means[:, np.newaxis]

  if test:
    Preds[idx_train] = np.nan

    return RMSE_mat(Preds, mat_test), MAE_mat(Preds, mat_test), W
  
  else:
    return RMSE_mat(Preds, mat_train), MAE_mat(Preds, mat_train), W

#### Etape e
Correction pour le facteur de rareté (fréquence inverse utilisateur). (1 point)

In [ ]:
def etape_e(mat_train, W, R_zeros, row_means, gamma = 5, test = False, mat_test = None, idx_train = None):
  """
  A l'inverse des précédentes, cette étape n'est pas symétriques en mode user-user et item-item

  Pour l'approche item-item, c'est assez naturel. On pondère la matrice de poids par les facteurs IDF car on veut directement avantager les items rares.

  Pour l'approche user-user en revanche le coefficient IDF appliqué aux utilisateurs n'a pas forcément de sens car on ne veut pas privilégier les utilisateurs avec le moins de votes différents.
  On implémente donc uniquement une solution item-item ici
  """
  IDF = np.log(mat_train.shape[1] / (np.sum(~np.isnan(mat_train), axis=1)) + 1)

  W = W * IDF

  Kappa = np.abs(W) @ ~np.isnan(mat_train)

  Kappa[Kappa == 0] = 1

  Preds = (W @ R_zeros) / Kappa + row_means[:, np.newaxis]

  if test:
    Preds[idx_train] = np.nan

    return RMSE_mat(Preds, mat_test), MAE_mat(Preds, mat_test)
  
  else:
    return RMSE_mat(Preds, mat_train), MAE_mat(Preds, mat_train)

#### Etape f
Un bonus de 2 points sera accordé si, à la question (b), vous évaluez correctement le biais entraîné par l'inclusion de la valeur observée dans le calcul de la moyenne pour cette même valeur prédite. Le maximum au TP demeure cependant de 20/20.


----
Pour cela on peut évaluer le biais avec une validation croisée leave-one-out qui permet d'avoir la meilleur estimation possible de la performance, sans perte de données tout en excluant la valeur testée du calcul de la moyenne de sa ligne et donc en soustayant le bais informatif de cette intersection entre les données réelles et les informations utilisées pour la prédiction. On compare à l'évaluation biaisée pour connaître le gain exact apporté par ce biais.

En pratique, la validation croisée leave-one-out est très longue (estimation 3.5h avec le code ci-dessous) donc on limite le nombre de replis à 1000, ce qui ne donnera pas une évaluation parfaite du biais mais permettra d'avoir une estimation.

In [ ]:
def etape_f_LOOCV(mat):
  """
  Cette fonction réalise le test non biaisé de l'étape.
  Son exécution est longue aussi on tuilise un version avec moins de replis qui donne une évaluation imparfaite mais plus rapide
  """
  list_notna = np.argwhere(~np.isnan(mat))

  RMSE = 0 #MAE et RMSE sont équivalente sur une unique prédiction

  for x,y in list_notna:
    value = mat[x ,y]
    mat[x ,y] = np.nan

    if np.nansum(mat[x, :]) == 0:

      RMSE += value #On prédit 0 s'il est impossible de faire une prédiction car c'était la seule valeur de la ligne, donc la RMSE sur cette unique valeur vaut value (car c'est toujours positif)

      mat[x ,y] = value
      continue

    row_means = np.nanmean(mat, axis=1)

    row_means[np.isnan(row_means)] = 0

    R = mat - row_means[:, np.newaxis]

    R_zeros = np.nan_to_num(R)

    norms = np.linalg.norm(mat, axis=1) #Les normes ne peuvent pas être nulles en leave one out dans notre cas car chaque ligne a au moins un élément
    W = (mat[x, :] @ mat.T) / (norms[x] * norms)

    W[x] = 0

    Kappa = np.abs(W) @ ~np.isnan(mat[:, y])

    if Kappa == 0 : Kappa = 1

    pred = (W @ R_zeros[:, y]) / Kappa + row_means[x]

    RMSE += np.abs(pred - value)

    mat[x ,y] = value
  
  return RMSE / len(list_notna)

In [ ]:
def etape_f(mat, n_folds):
  """
  Etape B avec validation croisée pour une évaluation non biaisée avec une par de données d'entraînement réglable
  """
  
  flat_index_notna = np.nonzero(~np.isnan(mat.ravel()))[0]

  assert len(flat_index_notna) % n_folds == 0, "n_folds should exactly divide 100k"

  np.random.shuffle(flat_index_notna)
  idx_split = np.split(flat_index_notna, n_folds)

  RMSE = 0
  MAE = 0
  
  for i in range(n_folds):
    idx_train = np.unravel_index(np.delete(idx_split, i, axis=0).flatten(), mat.shape)
    idx_test = np.unravel_index(idx_split[i], mat.shape)
    
    mat_train = mat.copy()
    mat_test = mat.copy()

    mat_train[idx_test] = np.nan
    mat_test[idx_train] = np.nan

    RMSE_fold, MAE_fold, _,_,_ = etape_b(mat_train, test = True, mat_test = mat_test, idx_train = idx_train)
    RMSE += RMSE_fold
    MAE += MAE_fold

  return RMSE / n_folds, MAE / n_folds

#### Etape g
Bonus 2 points : refaites les calculs (a) à (c) mais cette fois avec une validation croisée de 5 replis et comparez les résultats avec les calculs sans validation croisée. Le maximum au TP demeure cependant de 20/20.

In [ ]:
def etape_g(mat, n_folds=5):
  """
  Etape A à C avec validation croisée
  """
  flat_index_notna = np.nonzero(~np.isnan(mat.ravel()))[0]

  assert len(flat_index_notna) % n_folds == 0, "n_folds should exactly divide 100k"

  np.random.shuffle(flat_index_notna)
  idx_split = np.split(flat_index_notna, n_folds)

  RMSE_a = 0
  MAE_a = 0

  RMSE_b = 0
  MAE_b = 0

  RMSE_c = 0
  MAE_c = 0
  
  for i in range(n_folds):
    idx_train = np.unravel_index(np.delete(idx_split, i, axis=0).flatten(), mat.shape)
    idx_test = np.unravel_index(idx_split[i], mat.shape)
    
    mat_train = mat.copy()
    mat_test = mat.copy()

    mat_train[idx_test] = np.nan
    mat_test[idx_train] = np.nan

    RMSE_fold, MAE_fold = etape_a(mat_train, test = True, mat_test = mat_test, idx_train = idx_train)
    RMSE_a += RMSE_fold
    MAE_a += MAE_fold

    RMSE_fold, MAE_fold, W, R_zeros, row_means = etape_b(mat_train, test = True, mat_test = mat_test, idx_train = idx_train)
    RMSE_b += RMSE_fold
    MAE_b += MAE_fold

    RMSE_fold, MAE_fold, _ = etape_c(mat_train, W, R_zeros, row_means, test = True, mat_test = mat_test, idx_train = idx_train)
    RMSE_c += RMSE_fold
    MAE_c += MAE_fold

  return RMSE_a / n_folds, MAE_a / n_folds, RMSE_b / n_folds, MAE_b / n_folds, RMSE_c / n_folds, MAE_c / n_folds

### 2.3 Résultats et analyse

In [ ]:
def traitement(mat, approche, n_folds_bias_test):
  """
  Réalisation des tests sur l'ensemble des étapes selon l'approche voulue
  """
  if approche == 'user':
    MUI_mat = mat.to_numpy().copy()
    RMSE_list, MAE_list = [], []

    RMSE, MAE = etape_a(MUI_mat)
    RMSE_list.append(RMSE), MAE_list.append(MAE)
    RMSE, MAE, W, R_zeros, row_means = etape_b(MUI_mat)
    RMSE_list.append(RMSE), MAE_list.append(MAE)
    RMSE, MAE, W = etape_c(MUI_mat, W, R_zeros, row_means)
    RMSE_list.append(RMSE), MAE_list.append(MAE)
    RMSE, MAE, W = etape_d(MUI_mat, W, R_zeros, row_means)
    RMSE_list.append(RMSE), MAE_list.append(MAE)
    #Pas d'étape e en user-user
    RMSE_list.append(np.nan), MAE_list.append(np.nan)
    RMSE, MAE = etape_f(MUI_mat, n_folds_bias_test)
    RMSE_list.append(RMSE), MAE_list.append(MAE)
    RMSE_a, MAE_a, RMSE_b, MAE_b, RMSE_c, MAE_c = etape_g(MUI_mat)
    RMSE_list.extend((RMSE_a, RMSE_b, RMSE_c)), MAE_list.extend((MAE_a, MAE_b, MAE_c))

    return np.array([RMSE_list, MAE_list])

  elif approche == 'item':
    MIU_mat = mat.to_numpy().T.copy()
    RMSE_list, MAE_list = [], []

    RMSE, MAE = etape_a(MIU_mat)
    RMSE_list.append(RMSE), MAE_list.append(MAE)
    RMSE, MAE, W, R_zeros, row_means = etape_b(MIU_mat)
    RMSE_list.append(RMSE), MAE_list.append(MAE)
    RMSE, MAE, W = etape_c(MIU_mat, W, R_zeros, row_means)
    RMSE_list.append(RMSE), MAE_list.append(MAE)
    RMSE, MAE, W = etape_d(MIU_mat, W, R_zeros, row_means)
    RMSE_list.append(RMSE), MAE_list.append(MAE)
    RMSE, MAE = etape_e(MIU_mat, W, R_zeros, row_means)
    RMSE_list.append(RMSE), MAE_list.append(MAE)
    RMSE, MAE = etape_f(MIU_mat, n_folds_bias_test)
    RMSE_list.append(RMSE), MAE_list.append(MAE)
    RMSE_a, MAE_a, RMSE_b, MAE_b, RMSE_c, MAE_c = etape_g(MIU_mat)
    RMSE_list.extend((RMSE_a, RMSE_b, RMSE_c)), MAE_list.extend((MAE_a, MAE_b, MAE_c))

    return np.array([RMSE_list, MAE_list])

In [ ]:
def display_tab(results_uu, results_ii):
  """
  Tableau d'affichage des résulats de totues les étapes pour les deux approches
  """
  results_tab = pd.DataFrame(np.vstack((results_uu[0], results_ii[0], results_uu[1], results_ii[1])), columns=['base', 'mean', 'nearest neighbours', 'common votes', 'IDF', 'mean unbiased', 'base CV5', 'mean CV5', 'nearest neigbours CV5'], index=['RMSE user-user', 'RMSE item-item', 'MAE user-user', 'MAE item_item'])
  display(results_tab)

In [ ]:
#Traitement de toutes les étapes en approche utilisateurs-utilisateurs
#n_folds_bias_test influe sur le temps d'execution, avec 1000 l'execution prends une dizaine de minutes. Réduire pour une execution plus rapide
results_uu = traitement(MUI, 'user', n_folds_bias_test = 1000)

#Traitement de toutes les étapes en approche items-items
results_ii = traitement(MUI, 'item', n_folds_bias_test = 1000)

display_tab(results_uu, results_ii)

<ipython-input-115-8725f25d0d03>:2: RuntimeWarning: Mean of empty slice
  row_means = np.nanmean(mat_train, axis=1)


,base,mean,nearest neighbours,common votes,IDF,mean unbiased,base CV5,mean CV5,nearest neigbours CV5
RMSE user-user,1.015850,0.803798,0.803774,0.803774,NaN,0.926474,1.021160,0.939272,0.952457
RMSE item-item,1.012081,0.748296,0.734176,0.734176,0.727377,0.926649,1.018446,0.942490,0.949042
MAE user-user,0.808550,0.628008,0.625026,0.625026,NaN,0.730806,0.812796,0.739266,0.742983
MAE item_item,0.805072,0.587617,0.571074,0.571074,0.565593,0.730669,0.809063,0.740813,0.735346


Analyse des résultats et réponses :

Les résultats obtenus avec la méthode de base sont proches de notre baseline avec moyenne par item (sans cross validation) et un peu plus faible que notre meilleure baseline par valeur attendue (RMSE : 0.966809 et MAE : 0.782583). L'estimation de base étant une moyenne pondérée des voisins elle ne contient pas de régularisation par rapport aux informations de la ligne donc il n'est pas surprenant que les résultats soient faibles. On dépasse la baseline assez fortement grâce au gain important par l'ajout de la correction des moyennes Ensuite, les autres composantes apportent un gain plus modéré, d'environ 0.01 poitn d'erreur, voir nul pour l'amplification des utilisateurs avec votes communs.

L'estimation du biais induit par l'ajout de la valeur réelle dans la moyenne utilisée pour la prédiction à l'étape B est faite par une validation croisée à 1000 replis. L'estimation n'est pas parfaite (Il faudrait un leave-one-out pour cela mais c'est assez long à tourner) et une perte d'information susbsite liée à la diminution de la taille du dataset d'entraînement mais on peut estimer qu'elle reste raisonnable avec 1000 replis (1% des données retirées). Cette estimation exhibe une différence de RMSE de 0.12 à 0.18 (selon l'approche u-u ou i-i) et une différence de MAE 0.11 à 0.15 soit un gain d'environ 20% sur les erreurs uniquement due à un biais d'estimation.

On observe aussi naturellement une erreur plus grand avec la validation croisée sur les étapes A à C du fait du nombre de données réduit et du retrait du biais.

Si l'on compare les approches utilisateurs-utilisateurs et items-items on constate aussi que l'approche i-i est plus performante de manière assez notable dans toutes les évaluations biaisées après l'ajoute de la correction par la moyenne mais que ce gain est retiré avec une évaluation non biaisée. On peut penser que le fait qu'il y ait moins d'utilisateurs que d'items rend l'inclusion d'un biais plus simple dans une moyenne par une valeur ou aussi que les moyennes sont moins informatives au niveau utilisateur du fait d'une variance interne plus élevée.
